# Day 3 - Binary Diagnostic

>The submarine has been making some odd creaking noises, so you ask it to produce a diagnostic report just in case.
>
>The diagnostic report (your puzzle input) consists of a list of binary numbers which, when decoded properly, can tell you many useful things about the conditions of the submarine. The first parameter to check is the power consumption.
>
>You need to use the binary numbers in the diagnostic report to generate two new binary numbers (called the gamma rate and the epsilon rate). The power consumption can then be found by multiplying the gamma rate by the epsilon rate.
>
>Each bit in the gamma rate can be determined by finding the most common bit in the corresponding position of all numbers in the diagnostic report. For example, given the following diagnostic report:
>
>```
00100
11110
10110
10111
10101
01111
00111
11100
10000
11001
00010
01010
>```
>Considering only the first bit of each number, there are five 0 bits and seven 1 bits. Since the most common bit is 1, the first bit of the gamma rate is 1.
>
>The most common second bit of the numbers in the diagnostic report is 0, so the second bit of the gamma rate is 0.
>
>The most common value of the third, fourth, and fifth bits are 1, 1, and 0, respectively, and so the final three bits of the gamma rate are 110.
>
>So, the gamma rate is the binary number 10110, or 22 in decimal.
>
>The epsilon rate is calculated in a similar way; rather than use the most common bit, the least common bit from each position is used. So, the epsilon rate is 01001, or 9 in decimal. Multiplying the gamma rate (22) by the epsilon rate (9) produces the power consumption, 198.

In [20]:
from utils import read_input

diagnostics = read_input(3)

## Part 1

The rates are calculated by finding the most and least common bits in each position.

> Each bit in the gamma rate can be determined by finding the most common bit in the corresponding position of all numbers in the diagnostic report.

> The epsilon rate is calculated in a similar way; rather than use the most common bit, the least common bit from each position is used.

First, we need a way to transpose the diagnostics metrics so each value in the index 0 are in one list, values in index 1 in another and so on.

For the sake of learning, I'm implementing my own transpose function but in real use, I'd use [numpy's `matrix.transpose`](https://numpy.org/doc/stable/reference/generated/numpy.matrix.transpose.html).

In [19]:
def transpose(binary):
    transposed = []
    for reading in binary:
        for index, bit in enumerate(reading):
            try:
                transposed[index].append(bit) 
            except IndexError as e:
                transposed.append([bit])
    
    
    return transposed

Next, we create two new binaries for gamma and epsilon and count the most common and least common bits. Since there's only two options, 0 and 1, they will be exact opposites of each other.

In [109]:
from collections import namedtuple

Diagnostic = namedtuple('Diagnostic', ['gamma', 'epsilon'])

def count_bits(binary, tiebreaker):
    gamma = []
    epsilon = []
    
    for slot in binary:
        ones = slot.count('1')
        zeroes = slot.count('0')
        
        if ones > zeroes:
            gamma.append('1')
            epsilon.append('0')
        elif zeroes > ones:
            gamma.append('0')
            epsilon.append('1')
        else:
            gamma.append(tiebreaker)
            epsilon.append(tiebreaker)
            
    return Diagnostic(gamma=''.join(gamma), epsilon=''.join(epsilon))

Finally, we need to calculate the puzzle solution by multiplying the decimal values of the binaries.


In [73]:
transposed = transpose(diagnostics)
report = count_bits(transposed)

print(report)

result = int(report.gamma, 2) * int(report.epsilon, 2)

print(f'Solution: {result}')

Diagnostic(gamma='101110111100', epsilon='010001000011')
Solution: 3277364


## Part 2

>Next, you should verify the life support rating, which can be determined by multiplying the **oxygen generator rating** by the **CO2 scrubber rating**.
>
>Both the oxygen generator rating and the CO2 scrubber rating are values that can be found in your diagnostic report - finding them is the tricky part. Both values are located using a similar process that involves filtering out values until only one remains. Before searching for either rating value, start with the full list of binary numbers from your diagnostic report and consider just the first bit of those numbers. Then:
>
>- Keep only numbers selected by the bit criteria for the type of rating value for which you are searching. Discard numbers which do not match the bit criteria.
>- If you only have one number left, stop; this is the rating value for which you are searching.
>- Otherwise, repeat the process, considering the next bit to the right.
>
>The bit criteria depends on which type of rating value you want to find:
>
>- To find oxygen generator rating, determine the most common value (0 or 1) in the current bit position, and keep only numbers with that bit in that position. If 0 and 1 are equally common, keep values with a 1 in the position being considered.
>- To find CO2 scrubber rating, determine the least common value (0 or 1) in the current bit position, and keep only numbers with that bit in that position. If 0 and 1 are equally common, keep values with a 0 in the position being considered.
>
>For example, to determine the oxygen generator rating value using the same example diagnostic report from above:
>
>- Start with all 12 numbers and consider only the first bit of each number. There are more 1 bits (7) than 0 bits (5), so keep only the 7 numbers with a 1 in the first position: 11110, 10110, 10111, 10101, 11100, 10000, and 11001.
>-Then, consider the second bit of the 7 remaining numbers: there are more 0 bits (4) than 1 bits (3), so keep only the 4 numbers with a 0 in the second position: 10110, 10111, 10101, and 10000.
>-In the third position, three of the four numbers have a 1, so keep those three: 10110, 10111, and 10101.
>-In the fourth position, two of the three numbers have a 1, so keep those two: 10110 and 10111.
>-In the fifth position, there are an equal number of 0 bits and 1 bits (one each). So, to find the oxygen generator rating, keep the number with a 1 in that position: 10111.
>-As there is only one number left, stop; the oxygen generator rating is 10111, or 23 in decimal.
>
>Then, to determine the CO2 scrubber rating value from the same example above:
>
>-Start again with all 12 numbers and consider only the first bit of each number. There are fewer 0 bits (5) than 1 bits (7), so keep only the 5 numbers with a 0 in the first position: 00100, 01111, 00111, 00010, and 01010.
>-Then, consider the second bit of the 5 remaining numbers: there are fewer 1 bits (2) than 0 bits (3), so keep only the 2 numbers with a 1 in the second position: 01111 and 01010.
>-In the third position, there are an equal number of 0 bits and 1 bits (one each). So, to find the CO2 scrubber rating, keep the number with a 0 in that position: 01010.
>-As there is only one number left, stop; the CO2 scrubber rating is 01010, or 10 in decimal.
>
>Finally, to find the life support rating, multiply the oxygen generator rating (23) by the CO2 scrubber rating (10) to get 230.

In [123]:
def calculate_oxygen_rating(diagnostics):
    most_common_bits = count_bits(transpose(diagnostics), '1').gamma
    viable_ratings = diagnostics[:]
    
    index = 0
    while len(viable_ratings) > 1:
        compare_bit = most_common_bits[index]
        viable_ratings = [b for b in viable_ratings if b[index] == compare_bit]
        
        most_common_bits = count_bits(transpose(viable_ratings), '1').gamma
        index += 1
    
    return viable_ratings[0]

def calculate_co2_rating(diagnostics):
    most_common_bits = count_bits(transpose(diagnostics), '0').epsilon
    viable_ratings = diagnostics[:]
    
    index = 0
    while len(viable_ratings) > 1 or index > 11:
        compare_bit = most_common_bits[index]
        viable_ratings = [b for b in viable_ratings if b[index] == compare_bit]
        
        most_common_bits = count_bits(transpose(viable_ratings), '0').epsilon
        index += 1
    
    return viable_ratings[0]


In [117]:
test_input = ['00100', '11110', '10110', '10111', '10101', '01111', '00111', '11100', '10000', '11001', '00010', '01010']

print(calculate_oxygen_rating(test_input))
print(calculate_co2_rating(test_input))

print(f"Test solution: {int(calculate_oxygen_rating(test_input), 2) * int(calculate_co2_rating(test_input), 2)}")

10111
01010
Test solution: 230


In [124]:
oxygen = calculate_oxygen_rating(diagnostics)
co2 = calculate_co2_rating(diagnostics)

result = int(oxygen, 2) * int(co2, 2)

print(f'Result: {result}')

Result: 5736383
